# Análise RFM


 RFM Analysis, ou Análise RFM, é uma técnica de análise que é frequentemente utilizada no campo do marketing e da análise de clientes para segmentar e entender o comportamento dos clientes com base em três métricas principais:

1.   **<font color="#f4a261">Recência (R):</font>** Isso nos mostra quando um cliente fez sua última compra. Quanto mais recente a interação, mais valioso o cliente pode ser considerado, já que isso pode indicar um maior envolvimento e interesse atual.


2.   **<font color="#f4a261">Frequência (F):</font>** Aqui, olhamos para com que frequência um cliente compra.  Clientes que fazem compras com mais frequência podem ser considerados mais valiosos para a empresa.


3.   **<font color="#f4a261">Valor Monetário (M):</font>** Isso mostra o quanto um cliente gasta. Clientes que gastam mais dinheiro podem ser considerados mais lucrativos e valiosos para a empresa.


Ao combinar essas três métricas, é possível criar segmentos de clientes com diferentes perfis:


*   **<font color="#f4a261">Clientes Atrativos</font>** (High RFM | RFM Alto): São aqueles que fizeram uma compra recentemente, compram frequentemente e gastam muito dinheiro. Esses clientes são geralmente os mais valiosos.


*   **<font color="#f4a261">Clientes em Risco</font>** (Low R | Baixo R, High F | Alto F, High M | Alto M ou High R | Alto R, Low F | Baixo F, High M | Alto M): São aqueles que não fizeram uma compra recentemente, mas costumavam comprar frequentemente e gastavam muito dinheiro. Eles podem estar em risco de deixar de interagir com a empresa.

*   **<font color="#f4a261">Clientes Novos</font>** (High R | Alto R, High F | Alto F, Low M | Baixo M): São aqueles que fizeram uma compra recentemente, compram frequentemente, mas gastam relativamente pouco. Eles podem ser novos clientes explorando os produtos ou serviços.

*   **<font color="#f4a261">Clientes Inativos</font>** (Low RFM | Baixo RFM): São aqueles que não fizeram uma compra recentemente, não compram com frequência e gastam pouco dinheiro. Eles podem estar menos envolvidos com a empresa.




In [27]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from datetime import datetime


pio.templates.default = "plotly_white"
df = pd.read_csv("rfm_data.csv")

df.rename(columns={
  'CustomerID': 'ClienteID',
  'PurchaseDate': 'DataCompra',
  'TransactionAmount': 'Valor',
  'ProductInformation': 'produto_info',
  'OrderID': 'PedidoID',
  'Location': 'local'
}, inplace=True)
df


,ClienteID,DataCompra,Valor,produto_info,PedidoID,local
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


#  <font color="#e76f51">Cálculo de valores de RFM</font>

##  <font color="#f4a261">Recência  </font>

A recência é determinada calculando a diferença entre a data de compra mais recente e a data atual. O resultado desse cálculo nos fornece o número de dias desde a última compra do cliente.

In [30]:
# Convertendo 'DataCompra' para datetime:
df['DataCompra'] = pd.to_datetime(df['DataCompra'])

df['Recência'] = (datetime.now().date() - df['DataCompra'].dt.date).dt.days
df

AttributeError: Can only use .dt accessor with datetimelike values

AttributeError: Can only use .dt accessor with datetimelike values

## <font color="#f4a261">Frequência  </font>

A métrica de frequência é calculada individualmente para cada cliente. Para isso, os dados são agrupados com base no campo 'ClienteID', e o número de valores únicos no campo 'PedidoID' é contado. Esse processo permite determinar quantas compras cada cliente fez.

O resultado dessa contagem nos fornece a métrica de frequência, que representa o total de compras realizadas por cada cliente.

In [20]:
frequency_df = df.groupby('ClienteID')['PedidoID'].count().reset_index()
frequency_df.rename(columns={'PedidoID': 'Frequência'}, inplace=True)
df = df.merge(frequency_df, on='ClienteID', how='left')
df

,ClienteID,DataCompra,Valor,produto_info,PedidoID,local,Frequência
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,1
1,2188,2023-04-11,463.70,Product A,176819,London,1
2,4608,2023-04-11,80.28,Product A,340062,New York,1
3,2559,2023-04-11,221.29,Product A,239145,London,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,1
...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,1
996,6669,2023-06-10,941.50,Product C,987025,New York,1
997,8836,2023-06-10,545.36,Product C,512842,London,1
998,1440,2023-06-10,729.94,Product B,559753,Paris,1


## <font color="#f4a261">Valor Monetário  </font>

Os dados são agrupados com base no critério "ClienteID", e os valores da coluna "Valor" são somados. Isso resulta em um valor monetário único para cada cliente, refletindo o montante total de suas transações.

In [21]:
monetary_df = df.groupby('ClienteID')['Valor'].sum().reset_index()
monetary_df.rename(columns={'Valor': 'ValorMonetário'}, inplace=True)
df = df.merge(monetary_df, on='ClienteID', how='left')
df

,ClienteID,DataCompra,Valor,produto_info,PedidoID,local,Frequência,ValorMonetário
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,1,739.56
...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,1,759.62
996,6669,2023-06-10,941.50,Product C,987025,New York,1,941.50
997,8836,2023-06-10,545.36,Product C,512842,London,1,545.36
998,1440,2023-06-10,729.94,Product B,559753,Paris,1,729.94


#  <font color="#e76f51">Calculando as pontuações RFM </font>

## <font color="#f4a261">Definindo os critérios de pontuação para cada um dos valores do RFM  </font>

***Recência:***

Quanto mais recente a interação do cliente, maior será a pontuação.
>5 é a pontuação mais alta e representa uma recência menor (mais recente).

> 1 é a pontuação mais baixa e representa uma recência maior (menos recente).



***Frequência:***

Quanto mais o cliente interage ou compra, maior será a pontuação
> 1 é a pontuação mais baixa e representa uma frequência menor.

> 5 é a pontuação mais alta e representa uma frequência maior.


***Valor Monetário:***

Quanto mais dinheiro o cliente gasta, maior será a pontuação



> 1 é a pontuação mais baixa e representa um valor monetário menor.


> 5 é a pontuação mais alta e representa um valor monetário maior.



In [22]:
# Definição dos critérios:
recency_scores = [5, 4, 3, 2, 1]
frequency_scores = [1, 2, 3, 4, 5]
monetary_scores = [1, 2, 3, 4, 5]

# Cálculo das pontuações:
df['RecênciaScore'] = pd.cut(df['Recência'], bins=5, labels=recency_scores)
df['FrequênciaScore'] = pd.cut(df['Frequência'], bins=5, labels=frequency_scores)
df['MonetárioScore'] = pd.cut(df['ValorMonetário'], bins=5, labels=monetary_scores)

df

KeyError: 'Recência'

Para calcular as pontuações RFM, foi aplicada a função pd.cut() para segmentar os valores de recência, frequência e valor monetário em intervalos. Foram definidos 5 intervalos para cada valor e as pontuações correspondentes atribuidas a cada intervalo.

Após a adição das pontuações aos dados, é possível observar que elas são representadas como variáveis categóricas.

In [ ]:
print(f"RecênciaScore: {df['RecênciaScore'].dtype}")
print(f"FrequênciaScore: {df['FrequênciaScore'].dtype}")
print(f"MonetárioScore: {df['MonetárioScore'].dtype}")

##  <font color="#f4a261">Convertendo RFM scores para tipos numéricos </font>

 No entanto, para continuar utilizando as pontuações, é necessário converter essas variáveis categóricas em números inteiros.

In [ ]:
print(f"RecênciaScore antes da conversão: {df['RecênciaScore'].dtype}")
print(f"FrequênciaScore antes da conversão: {df['FrequênciaScore'].dtype}")
print(f"MonetárioScore antes da conversão: {df['MonetárioScore'].dtype}")
print()

# Conversão dos scores em inteiros:
df['RecênciaScore'] = df['RecênciaScore'].astype(int)
df['FrequênciaScore'] = df['FrequênciaScore'].astype(int)
df['MonetárioScore'] = df['MonetárioScore'].astype(int)

print()
print(f"RecênciaScore depois da conversão:  {df['RecênciaScore'].dtype}")
print(f"FrequênciaScore depois da conversão:  {df['FrequênciaScore'].dtype}")
print(f"MonetárioScore depois da conversão:  {df['MonetárioScore'].dtype}")

# <font color="#e76f51">Segmentação do valor do RFM </font>

Cálculo da pontuação final do RFM  (Recency-Frequency-Monetary) |  (Recência-Frequência-Monetário) combinando as pontuações individuais de recência, frequência e valor monetário para cada cliente.

Para calcular a pontuação RFM, foram adicionadas as pontuações obtidas para essas variáveis. Por exemplo, se um cliente tiver uma pontuação de recência de 3, uma pontuação de frequência de 4 e uma pontuação monetária de 5, a sua pontuação RFM será 12.

In [ ]:
# Calculando a pontuação final pela combinação das pontuações individuais:
df['RFM_Score'] = df['RecênciaScore'] + df['FrequênciaScore'] + df['MonetárioScore']

scores_data = {
    'ClienteID' : df['ClienteID'],
    'RecênciaScore' : df['RecênciaScore'],
    'FrequênciaScore' : df['FrequênciaScore'],
    'MonetárioScore' : df['MonetárioScore'],
    'RFM_Score' : df['RFM_Score'],
}

scores_df = pd.DataFrame(scores_data)
scores_df

Criação dos segmentos RFM com base no score RFM calculado anteriormente (RFM_Score).

A função pd.qcut() é usada para dividir os scores RFM em intervalos de forma a garantir que cada segmento contenha aproximadamente a mesma quantidade de clientes.


> data['RFM_Score'] = coluna dos scores RFM que serão segmentados.


> q = número de segmentos.



> 'labels=labels_set' = rótulos atribuidos a cada segmento.









In [ ]:
# Criando os segmentos RFM:
labels_set = ['Valor-Baixo', 'Valor-Médio', 'Valor-Alto']
df['ValorSegmento'] = pd.qcut(df['RFM_Score'], q=3, labels=labels_set)



   O resultado dessa operação é uma nova coluna chamada 'ValorSegmento' no DataFrame df, que contém os rótulos dos segmentos aos quais cada cliente pertence, com base no seu score RFM.

In [ ]:
df

# Distribuição por segmentos <font color="#2a9d8f">(View)</font>

In [ ]:
# Distribuição dos segmentos RFM:
segment_counts = df['ValorSegmento'].value_counts().reset_index()
segment_counts.columns = ['ValorSegmento', 'Contagem']

pastel_colors = px.colors.qualitative.Pastel

fig_segment_dist = px.bar(segment_counts, x='ValorSegmento', y='Contagem',
                          color='ValorSegmento', color_discrete_sequence=pastel_colors,
                          title="Distribuição dos segmentos RFM:")

fig_segment_dist.update_layout(xaxis_title='RFM Valor Segmento',
                              yaxis_title='Contagem',
                              showlegend=False)

fig_segment_dist.show()

# Segmentos de clientes RFM



 Os segmentos de clientes RFM representam diferentes grupos de clientes com base em suas pontuações RFM. O que foi feito anteriormente foi a criação de segmentos de "valor" RFM, que categorizam os clientes em grupos de "baixo valor", "valor médio" e "alto valor" com base em suas pontuações.

Agora, serão criados segmentos de "clientes" RFM. Isso significa dividir os clientes em grupos com base em suas pontuações RFM. Esses grupos ajudarão a entender melhor o comportamento dos clientes em termos de quando compraram (recência), com que frequência compram (frequência) e quanto gastam (monetário). Cada grupo representará diferentes tipos de clientes, permitindo uma análise mais granular das características RFM globais do cliente. Isso é útil para entender o valor relativo dos clientes com base nesses três aspectos-chave.

In [ ]:
# Nova coluna para os segmentos RFM do cliente:
df['RFM_Segmento_Cliente'] = ''

 Esta coluna será usada para armazenar os segmentos de clientes com base na pontuação RFM.

In [ ]:
df.loc[df['RFM_Score'] >= 9, 'RFM_Segmento_Cliente'] = 'Campeões'
df.loc[(df['RFM_Score'] >= 6) & (df['RFM_Score'] < 9), 'RFM_Segmento_Cliente'] = 'Potencial de fidelidade'
df.loc[(df['RFM_Score'] >= 5) & (df['RFM_Score'] < 6), 'RFM_Segmento_Cliente'] = 'Clientes em risco'
df.loc[(df['RFM_Score'] >= 4) & (df['RFM_Score'] < 5), 'RFM_Segmento_Cliente'] = "Não pode perder"
df.loc[(df['RFM_Score'] >= 3) & (df['RFM_Score'] < 4), 'RFM_Segmento_Cliente'] = "Perdido"

df[['ClienteID', 'RFM_Segmento_Cliente']]

# Análise RFM <font color="#2a9d8f">(View)</font>




> ## Análise da distribuição de clientes em diferentes segmentos de clientes RFM dentro de cada segmento de valor.



Gráfico Treemap para visualizar a distribuição de segmentos de cliente com base nos valores de 'ValorSegmento' e 'RFM_Segmento_Cliente', destacando as contagens de ocorrências e as cores correspondentes.

In [ ]:
# Agrupando os dados e contando o número de ocorrências para cada combinação de segmentos:
segment_product_counts = df.groupby(['ValorSegmento', 'RFM_Segmento_Cliente']).size().reset_index(name='Contagem')

# Ordenando os dados pela contagem em ordem decrescente:
segment_product_counts = segment_product_counts.sort_values('Contagem', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['ValorSegmento', 'RFM_Segmento_Cliente'],
                                         values='Contagem',
                                         color='ValorSegmento', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM_Segmento_Cliente por Valor')
fig_treemap_segment_product.show()



> ## Análise da <font color="#a2d2ff">distribuição dos valores RFM </font> no segmento dos Campeões.



Permite visualizar como as variáveis estão distribuídas entre os clientes mais valiosos.

In [ ]:
# Filtrando para incluir apenas os clientes no segmento "Campeões":
champions_segment = df[df['RFM_Segmento_Cliente'] == 'Campeões']

# Objeto go.Figure() do Plotly:
fig = go.Figure()

 Três box plots estão sendo adicionados ao objeto fig. Cada go.Box() cria um box plot para uma variável específica.



> Para cada Box, o argumento y é usado para fornecer os valores que serão plotados no eixo vertical do gráfico.



> O argumento 'name' define o nome do gráfico de caixa, que aparecerá na legenda do gráfico.





In [ ]:
fig.add_trace(go.Box(y=champions_segment['RecênciaScore'], name='Recência'))
fig.add_trace(go.Box(y=champions_segment['FrequênciaScore'], name='Frequência'))
fig.add_trace(go.Box(y=champions_segment['MonetárioScore'], name='Monetário'))


fig.update_layout(title='Distribuição dos valores RFM dentro do segmento de clientes "Champions"',
                  yaxis_title='Valores RFM',
                  showlegend=True)

fig.show()



> ## Análise da  <font color="#a2d2ff">correlação das pontuações  </font> de recência, frequência e monetário no segmento dos campeões.



In [ ]:
# Calculando a matriz de correlação:
correlation_matrix = champions_segment[['RecênciaScore', 'FrequênciaScore', 'MonetárioScore']].corr()

fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlação')))

fig_heatmap.update_layout(title='Matriz de Correlação dos valores RFM dentro do segmento dos campeões.')

fig_heatmap.show()



 > ## Número de clientes em todos os segmentos



Comparação da contagem de segmentos de clientes com base nas classificações RFM.

In [ ]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

# Contando quantos clientes existem em cada segmento de clientes:
# [Campeões, Potencial de fidelidade, Clientes em risco, Não pode perder, Perdido]
segment_counts = df['RFM_Segmento_Cliente'].value_counts()

fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])


champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Campeões' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

fig.update_layout(title='Comparação dos Segmentos RFM',
                  xaxis_title='Segmentos RFM',
                  yaxis_title='Número de Clientes',
                  showlegend=False)

fig.show()




> ## Comparação dos scores de Recência, Frequência e Valor Monetário (RFM) entre diferentes segmentos de clientes




In [ ]:
# Calculando as médias de pontuação (score) de Recência, Frequência e Valor Monetário para cada segmento de clientes:
segment_scores = df.groupby('RFM_Segmento_Cliente')['RecênciaScore', 'FrequênciaScore', 'MonetárioScore'].mean().reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=segment_scores['RFM_Segmento_Cliente'],
    y=segment_scores['RecênciaScore'],
    name='Recência Score',
    marker_color='rgb(158,202,225)'
))

fig.add_trace(go.Bar(
    x=segment_scores['RFM_Segmento_Cliente'],
    y=segment_scores['FrequênciaScore'],
    name='Frequência Score',
    marker_color='rgb(94,158,217)'
))

fig.add_trace(go.Bar(
    x=segment_scores['RFM_Segmento_Cliente'],
    y=segment_scores['MonetárioScore'],
    name='Monetário Score',
    marker_color='rgb(32,102,148)'
))

fig.update_layout(
    title='Comparação dos valores RFM dos segmentos de cliente e as variáveis RFM.',
    xaxis_title='Segmentos de Cliente RFM',
    yaxis_title='Pontuação da Variável',
    barmode='group',
    showlegend=True
)

fig.show()
